# Домашнее задание № 10. Генерация текста

### Задание 1 (10 баллов).

В семинаре мы работали с датасетами инструкций alpaca и dolly. Они англоязычные. В домашке вам нужно создать аналогичный датасет на русском языке и обучить аналогичную модель на этом датасете.
В качестве итогового результата у вас должна получится модель, которая может связно отвечать на русскоязычные инструкции на русском языке. Приведите как минимум три разных примера. Правильность ответов не так важна, так как вы скорее всего будете использовать небольшие модели, но текст должен быть не рандомным.

Русскоязычный датасет инструкций должен быть больше 5 тысяч примеров. Он может быть основнован на alpaca/dolly (например, вы можете просто прогнать все через переводную модель, которая была на семинаре, или даже google translate). Или вы можете придумать способ создать аналогичный датасет каким-то другим способом (переделать открытые датасеты с помощью правил). Датасет может быть не уникальным, можно скооперироваться с одногруппниками и сделать один датасет на всех.

Вы можете попробовать дообучать любую небольшую decoder-only модель. Скорее всего лучше всего будут работать модели, изначально обученные на русском языке (rugpt например). Но возможно даже модели вроде opt можно будет дообучить на русскоязычных инструкциях.

Это задание гораздно менее определенное, по сравнению с предыдущими. Поэтому не стесняйтесь задавать дополнительные вопросы в чате или лично, если у вас возникнут трудности.

In [1]:
# !pip install torch

In [2]:
# !pip install accelerate==0.21.0

In [3]:
!pip install datasets zstandard jsonlines transformers==4.30

In [4]:
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer

In [5]:
from datasets import load_dataset

dataset = load_dataset("IlyaGusev/ru_turbo_alpaca")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for IlyaGusev/ru_turbo_alpaca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/ru_turbo_alpaca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `d

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'],
        num_rows: 29822
    })
})

In [7]:
for i in range(0, 9):
  print(dataset['train'][i]['instruction'])

Опишите, как сделать горшок из глины.
Опиши процесс изготовления торта. Укажи ингредиенты, необходимые для этого.
Напиши аналог данной пословицы на современный лад: "Не говори гоп, пока не перепрыгнешь".
Представьте, что вы занимаетесь дизайном главной страницы для нового сайта. Опишите, что вы хотели бы видеть на этой странице.
Напиши рассказ о своем детстве и самом смешном инциденте, который произошел с тобой.
Напишите краткую историю о двух лучших друзьях.
Придумай название нового магазина.
Напишите рекламное объявление о продукте или услуге, используя привлекательный язык и убедительные доводы.
Предложите тему для диссертации в области обработки естественного языка.


In [8]:
import json

In [9]:
subset_list = []

for i in range(0, 9999):
    subset_dict = {
        'instruction': dataset['train'][i]['instruction'],
        'input': dataset['train'][i]['input'],
        'output': dataset['train'][i]['output']
    }
    subset_list.append(subset_dict)

with open("ru_alpaca.json", "w") as json_file:
    json.dump(subset_list, json_file, indent=2)

In [10]:
# data_alpaca = json.load(open('ru_alpaca.json'))

In [11]:
# del data_alpaca

In [12]:
# len(data_alpaca)

In [13]:
# data_alpaca[:2]

In [14]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Ниже будут инструкции для решения задачи, а также текст, содержащий больше контекста. "
        "Напиши отвечающий запросу ответ.\n\n"
        "### Инструкции:\n{instruction}\n\n### Контекст:\n{input}\n\n### Ответ:"
    ),
    "prompt_no_input": (
        "Ниже будут инструкции для решения задачи. "
        "Напиши отвечающий запросу ответ.\n\n"
        "### Инструкции:\n{instruction}\n\n### Ответ:"
    ),
}

In [15]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [16]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

In [17]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = json.load(open(data_path))

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")

In [19]:
train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path="ru_alpaca.json")
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [20]:
train_args = transformers.TrainingArguments(learning_rate=1e-4,
                 num_train_epochs=2,
                 per_device_train_batch_size=2,
                 gradient_accumulation_steps=1,
                 evaluation_strategy='no',
                 weight_decay=0.1,
                 warmup_ratio=0.03,
                 lr_scheduler_type="cosine",
                 save_strategy='no',
                 logging_steps=1000,
                 output_dir="rugpt_instruct_ft")

In [21]:
trainer = Trainer(model=model,
                 tokenizer=tokenizer,
                 args=train_args,
                 train_dataset=train_dataset,
                 eval_dataset=None,
                 data_collator=data_collator)

In [22]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1000,2.627500
2000,2.589400
3000,2.527300
4000,2.459800
5000,2.392100
6000,1.608800
7000,1.572200
8000,1.544000
9000,1.531100
10000,1.511900


TrainOutput(global_step=10000, training_loss=2.0364072265625, metrics={'train_runtime': 1294.862, 'train_samples_per_second': 15.444, 'train_steps_per_second': 7.723, 'total_flos': 1402499271168000.0, 'train_loss': 2.0364072265625, 'epoch': 2.0})

In [23]:
trainer.save_model('rugtp_ft')

In [24]:
def predict_for_instruction(instruction, text, model, tokenizer):
    text = text.replace('\n', ' ')
    prompt = ("Ниже будут инструкции для решения задачи, а также текст, содержащий больше контекста. "
              "Напиши отвечающий запросу ответ.\n\n"
              f"### Инструкции:\n{instruction}\n\n### Контекст:\n{text}\n\n### Ответ:")

    inputs = tokenizer([prompt], return_tensors="pt", padding=True)

    inputs = {key: tensor.to(model.device) for key, tensor in inputs.items()}

    with torch.no_grad():
        output_sequences = model.generate(
            max_length=2000,
            early_stopping=True,
            num_return_sequences=1,
            **inputs
        )

    output = tokenizer.batch_decode(output_sequences[:, len(tokenizer.encode(prompt)):], skip_special_tokens=True)
    return output[0]


In [25]:
text = """

"""

In [26]:
instruction = "Напиши краткую рекомендацию о том, как приготовить пиццу."
predict_for_instruction(instruction, text, model, tokenizer)

'Для приготовления пиццы нужно взять тесто, раскатать его на плоской поверхности и нанести на нее соус. Выложить тесто на противень и запекать в духовке при температуре 200 градусов в течение 10-15 минут.'

In [27]:
text = """
Штирлиц играл в карты и проигрался. Но Штирлиц умел делать хорошую мину при плохой игре. Когда Штирлиц покинул компанию, мина сработала.

Идет Штирлиц ночью по городу, навстречу мужик бородатый и в чалме.
— Ах, будь он не Ладен - подумал Штирлиц.

Штирлиц и Мюллер ездили по очереди на танке. Очередь редела, но не расходилась...

Штирлиц стрелял вслепую. Слепая испугалась и побежала скачками, но качки быстро отстали.

Подвыпившие Штирлиц и Мюллер вышли из бара.
— Давайте снимем девочек, — предложил Штирлиц.
— У вас очень доброе сердце, — ответил Мюллер. — Но пусть все-таки повисят до утра.
"""

In [28]:
instruction = "Выбери самый смешной анекдот."
predict_for_instruction(instruction, text, model, tokenizer)

'Штирлиц был очень рад, что смог выиграть игру. Он был рад, что смог выиграть эту партию. Но когда он вернулся домой, он был очень расстроен тем, что проиграл. Он был рад, что смог выиграть эту партию.'

In [29]:
text = """

"""

In [34]:
instruction = "Почему Земля круглая?"
predict_for_instruction(instruction, text, model, tokenizer)

'Земля круглая - это эллиптическая форма, в которой центр тяжести находится между Землей и Солнцем.'

In [36]:
instruction = "Придумай анекдот."
predict_for_instruction(instruction, text, model, tokenizer)

'Встретились два студента в университете. Один из них спросил: "Как тебе наша общая задача?" \nВторой студент ответил: "Да, это наша общая задача."'